# JSON資料型態

### 看看wiki怎麼說

- https://zh.wikipedia.org/zh-tw/JSON

```
{"time":"2020-07-01", "data": [1, 2, 3, 4, 5]}
```

### 空氣品質應用
- 資料來源：https://data.gov.tw/dataset/40448


### JSON格式初探
- https://zh.wikipedia.org/zh-tw/JSON

In [ ]:
import requests

In [ ]:
#url = 'http://opendata.epa.gov.tw/ws/Data/AQI/?$format=json' # 已失效
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_432?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=ImportDate%20desc&format=JSON'

In [ ]:
r = requests.get(url)

In [ ]:
r

In [ ]:
data = r.json()

In [ ]:
data['records']

In [ ]:
records = data['records']

In [ ]:
for item in records:
    if item['county']=='臺南市':
        print(item['county'], item['sitename'], item['pm2.5'])

- 小試身手: 把高雄站點的AQI, 硫化物, 還有GPS座標印出來

In [ ]:
# 把程式填入這邊
for item in records:
    if item['county']=='臺南市':
        print(item['county'], item['sitename'], item['pm2.5'], item['latitude'], item['longitude'])


### 進一步找出 全台有超標的站，並列出來
- AQI 是否超標，請參照以下的資訊
- ref: https://airtw.epa.gov.tw/CHT/Information/Standard/AirQualityIndicator.aspx

In [ ]:
for item in records:
    aqi = float(records[0]['aqi'])


In [ ]:
# 把程式填入這邊
for item in records:
    aqi = float(item['aqi'])
    if aqi>50:
        print(item['county'], item['sitename'], item['aqi'], item['latitude'], item['longitude'])


### 在地圖上把所有的站畫出來

In [ ]:
# 在colab用底下的安裝指令
!pip install geopy

In [ ]:
#引入相關套件，並建立取得GPS座標的function
import folium
import geopy
from geopy.geocoders import Nominatim

def get_gps_coordinates(location_name):
    """
    取得指定地名的 GPS 座標。

    Args:
        location_name (str): 地名。

    Returns:
        tuple: (緯度, 經度)，如果找不到則返回 None。
    """
    try:
        geolocator = Nominatim(user_agent="my_geocoder")
        location = geolocator.geocode(location_name)
        if location:
            return location.latitude, location.longitude
        else:
            return None
    except Exception as e:
        print(f"發生錯誤：{e}")
        return None


In [ ]:
# 把程式填入這邊

# 使用 geocoder 取得特定住址的 GPS 座標
location = get_gps_coordinates('台灣')
print(location)


m = folium.Map(location=location, zoom_start=10)

for item in records:
    try:
        name = item['sitename']  #站名
        lat = item['latitude']  #緯度
        lng = item['longitude']  #經度
        aqi = int(item['aqi'])
        info = '[%s] %s' %(name, aqi)
        folium.Marker([ float(lat), float(lng)],
                      tooltip=info,
                     icon=folium.Icon(color='blue')
                     ).add_to(m)
        
    except Exception as e:
        print(e.args)    

In [ ]:
m

### 畫出地圖，並且把超標的畫紅色、沒超標的畫綠色

### 把成果下載下來欣賞

In [ ]:
# 寫下你的程式
m.save('TW-air.html')